In [2]:
import tweepy
import time

In [3]:
import smtplib

In [4]:
import os
print(os.getcwd())

C:\Users\PC\Documents\GitHub\Moe_Antar


In [5]:
os.chdir("C:\\Users\\PC\\Documents\\GitHub\\Moe_Antar\\Twitter Analysis with LDA")

In [6]:
from tweepy import OAuthHandler

In [7]:
consumer_key = 'lvxLcrv96o2LLBUjd805anFEe'
consumer_secret = 'UUftHUPiI0inirhZhaDln1NchRzMz4qhVeHwWYf3HhcEdGBDhK'
access_token = '826992578075648002-AbfhDdp9hVKZgQV4dnJd6obe3h9do9R'
access_secret = 'myNE88Yf6MIwcvEdoqNXH67WgqvfjaCaCtVj2IbUIouYC'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [8]:
# SET THE EMAIL ACCOUNT THAT WILL SEND THE NOTIFICATION HERE

In [9]:
#EMail script adapted from http://stackabuse.com/how-to-send-emails-with-gmail-using-python/
import smtplib

gmail_user = ''  
gmail_password = ''

try:  
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    print('Login Attempt Complete. No Errors So Far')
except:  
    print('Failed to Log in to Email Service...')

Login Attempt Complete. No Errors So Far


In [10]:
#EMail script adapted from http://stackabuse.com/how-to-send-emails-with-gmail-using-python/
# function that will send whatever you want to an email address you specify
def send_email(recipient_list=['moe.antar@hotmail.com'],message="Hello",email_subject='Listener Notification'):
    import smtplib

    gmail_user = 'moe.python.app@gmail.com'  
    gmail_password = 'testing_python_app'
    
    sent_from = gmail_user  
    to = recipient_list 
    subject = email_subject 
    body = message
    
    email_text = """\  
    From: %s  
    To: %s  
    Subject: %s

    %s
    """ % (sent_from, ", ".join(to), subject, body)

    try:  
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, email_text)
        server.close()

        print('Email sent!')
    except:  
        print('Failed to send Email')

In [11]:
# Test the script's ability to send notifications
send_email(recipient_list=['moe.antar@hotmail.com'],
           message="Twitter Listener Has Crashed due to Error : Internet Connection Failure /n Tweets Collected So far: 496",
           email_subject="Moe's Listener Notification: Twitter Listener Has Crashed  " )

Email sent!


In [12]:
# Initializing a listener class that streams from  Twitter
class StdOutListener(tweepy.StreamListener):
    
    #The init function of a class allows us to have variables that  are set to specific values when an instance of the class is intiialized
    # We can then change these attributes from outside the class by calling CLASSNAME.ATTRIBUTENAME = something
    def __init__(self, api=None):
        super(StdOutListener, self).__init__()
        # Initialize the customization attributes that we want to be able to change from the outside 
        self.num_tweets = 0
        self.tweet_limit = 100
        self.file_label = "streamed_tweets"
        self.recipients = 'moe.antar@hotmail.com'
        self.notification_interval = 1000
        self.allow_email = True
        self.allow_progress_email = True
        self.progress_email_interval = 100000
        
    def set_tweet_limit(self,tweet_quantity):
        self.tweet_limit = tweet_quantity
        
    def set_file_label(self,file_label):
        self.file_label = file_label
        
    def set_email_permission(self,allow_email):
        self.allow_email = True
        
    def set_email_recipients(self,email_recipients):
        self.recipients = email_recipients
        
    def set_console_notification_interval(self,console_notification_interval):
        self.notification_interval = console_notification_interval
        
    def set_progress_email_permission(self,progress_email_permission):
        self.allow_progress_email = progress_email_permission
        
    def set_progress_email_interval(self,progress_email_interval):
        self.progress_email_interval = progress_email_interval
    
    def on_data(self, data):
        # Based on the interval chosen by the user, let the user know how many tweets have been collected so far
        if self.num_tweets % self.notification_interval == 0:
            print("(%d) Tweets Collected So far" % self.num_tweets)
            
        # If the user has allowed emails and has chosen to recieve progress emails
        if self.allow_email == True and self.allow_progress_email == True and self.num_tweets % self.progress_email_interval == 0 and self.num_tweets != 0 :
            try:
                send_email(recipient_list=self.recipients,message="(%d) Tweets Collected So Far" %self.num_tweets , 
                           email_subject="Listener Progress Notification")
                print("Progress Email Sent: (%d) tweets collected " % self.num_tweets)
            except:
                print("Failed to send progress email")
                
            
        # Increment the number of tweets collected by 1
        self.num_tweets += 1
        
        # This is the main script that collects the tweets
        # It will terminate when it hits the tweets limit
        if self.num_tweets < self.tweet_limit:
            try:
                with open('%s.json' %self.file_label, 'a') as f:
                    f.write(data)
                    return True
            except KeyboardInterrupt:
                print("Keyboard Interrupt: Ending Stream")
            except BaseException as e:
                print(str(e))
            return True
        else:
            print("Tweet Limit Reached: (%d) .... Closing Stream " % self.num_tweets)
            return False
        
    def on_error(self, status):
        print(status)

In [13]:
# Set up function that will initialize this class, and add the constraints we want , 
#as well as filter the feed for the subjects we're interested int

def collect_tweets_from_stream(subjects: object,
                               max_tweets: object, 
                               file_label: object = "streamed_tweets",
                               auth: object = auth,
                               email_recipients: object = ['moe.antar@hotmail.com'] , 
                               console_notification_interval: object = 10000 , 
                               allow_email: object = True,
                               progress_email_permission:object = True,
                               progress_email_interval: object = 500000
                               ) -> object:
    
    my_listener = StdOutListener()
    my_listener.set_tweet_limit(max_tweets)
    my_listener.set_file_label(file_label)
    my_listener.set_email_recipients(email_recipients)
    my_listener.set_console_notification_interval(console_notification_interval)
    my_listener.set_email_permission(allow_email)
    my_listener.set_progress_email_permission(progress_email_permission)
    my_listener.set_progress_email_interval(progress_email_interval)
    
    stream = tweepy.Stream(auth, my_listener)
    
    print("Beginning Stream")
    print("Will collect until %d tweets are reached" %max_tweets )
    print("Output file will be called %s .json " % file_label)
    if allow_email == True :
        print("Email Notifications Are Enabled")
        print("Progress Emails Will Be Sent To : %s" %email_recipients  )

    
    
    try:
        stream.filter(track=subjects)
    except KeyboardInterrupt as e:
        print("Keyboard Interrupt: Stopping Stream")
    except:
        print("Listening function: Termination Caused by Unknown Error")

In [21]:
#Competitor listener
##You need to set the following inputs (examples shown):

### What to search for
#### subjects = ["word1","word"]

### After How many tweets to stop listening
#### max_tweets = 1000

### What to name the output JSON file
#### file_label = "bestbuy tweets"

### What authentication tokens to use (The Auth variable should be declared earlier in the script)
#### auth = auth

### Whether to allow email notification
#### allow_email = True

### If using email notification , who to send the notifications to (if not using it, enter some placeholder email like ["nothing@nothing.com"] and make sure to set allow_email to false)
#### email_recipients = ["thisemail@whatever.com","thatemail@whatever.com"]

### How often to send a notification to the console and (optionally) to email that says how many tweets have been collected so far
#### console notification interval = 10000  

### Should It send progress emails every n tweets to let you know it has collected this much tweets?
#### progress_email_permission = True

### How often to send progress emails?
#### progress_email_interval = 100,000

In [22]:
# collect_tweets_from_stream(
# subjects=['@amazon','@Walmart','@WalmartCanada',"@VisionsDeals","@visionsburnaby",'@BestBuyCanHelp','@BBYCanadaDeals','@WalmartTech',
# '@samsung','@SamsungMobile','@SamsungMobileUS','@SamsungTV','@SamsungUS','@Sony','@SonyElectronics','@PlayStation','@Xbox','@XboxCanada','@NintendoSwitchC','@NintendoAmerica','@Apple',
# '@AppleSupport','@tim_cook','samsung','apple',
# 'nintendo','horizon zero dawn','ubisoft'],
# max_tweets=100,
# file_label="test100tweets",
# auth=auth,
# allow_email = True,
# email_recipients= ["moe.antar@hotmail.com"],
# console_notification_interval= 10,
# progress_email_permission = True,
# progress_email_interval= 50
# )

Beginning Stream
 Will collected until 100 tweets are reached
Output file will be called test100tweets .json 
Email Notifications Are Enabled
Progress Emails Will Be Sent To : ['moe.antar@hotmail.com']


(0) Tweets Collected So far


(10) Tweets Collected So far


(20) Tweets Collected So far


(30) Tweets Collected So far


...

(60) Tweets Collected So far


(70) Tweets Collected So far


(80) Tweets Collected So far


(90) Tweets Collected So far


Tweet Limit Reached: (100) .... Closing Stream 


In [14]:
def main_function(
subjects=None,
max_tweets=100,
file_label="test100tweets",
auth=auth,
allow_email = True,
email_recipients= ["moe.antar@hotmail.com"],
console_notification_interval= 10,
progress_email_permission = True,
progress_email_interval= 50     
            
):
    if subjects is None:
        subjects = ['@amazon', '@Walmart', '@WalmartCanada', "@VisionsDeals", "@visionsburnaby", '@BestBuyCanHelp',
                    '@BBYCanadaDeals', '@WalmartTech',
                    '@samsung', '@SamsungMobile', '@SamsungMobileUS', '@SamsungTV', '@SamsungUS', '@Sony',
                    '@SonyElectronics', '@PlayStation', '@Xbox', '@XboxCanada', '@NintendoSwitchC', '@NintendoAmerica',
                    '@Apple',
                    '@AppleSupport', '@tim_cook', 'samsung', 'apple',
                    'nintendo', 'horizon zero dawn', 'ubisoft']
    attempt_count = 0
    while attempt_count < 3:
        print("Main Function Attempting To Start")
        try:
            attempt_count = attempt_count + 1 
            print("Attempt # %d of 2" %attempt_count)
            collect_tweets_from_stream(
            subjects=subjects,
            max_tweets=max_tweets,
            file_label=file_label,
            allow_email = allow_email,
            auth=auth,
            email_recipients=email_recipients,
            console_notification_interval= console_notification_interval,
            progress_email_permission = progress_email_permission,
            progress_email_interval= progress_email_interval
            )
            return 
            
        except KeyboardInterrupt as e:
            print("Keyboard interrupt.. Closing Stream")
            return
        except: 
            print("Stream Interrupt Due to Unkown Cause: Will Attempt to Restart After 30 Seconds")
            time.sleep(30)
            attempt_count = attempt_count + 1

In [15]:
main_function()

Main Function Attempting To Start
Attempt # 1 of 2
Beginning Stream
Will collect until 100 tweets are reached
Output file will be called test100tweets .json 
Email Notifications Are Enabled
Progress Emails Will Be Sent To : ['moe.antar@hotmail.com']


(0) Tweets Collected So far


(10) Tweets Collected So far


(20) Tweets Collected So far


(30) Tweets Collected So far


...

(60) Tweets Collected So far


(70) Tweets Collected So far


(80) Tweets Collected So far


(90) Tweets Collected So far


Tweet Limit Reached: (100) .... Closing Stream 
